In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.io import loadmat
import networkx as nx
import numpy as np
import os 
from scipy.sparse import csgraph
from scipy.linalg import eigh
import gudhi as gd
import gudhi.representations as tda
from sklearn.preprocessing import MinMaxScaler
from tensorflow import random_uniform_initializer as rui
from sklearn.model_selection import train_test_split
import tensorflow_addons  as tfa



In [2]:
def HKS(egvec, egval, t):
    return np.square(egvec).dot(np.diag(np.exp(-t*egval))).sum(axis=1)

In [3]:
def extended_persistance(adj, values):
    
    num_vertices = adj.shape[0]
    (xs, ys) = np.where(np.triu(adj))
    st = gd.SimplexTree()
    for i in range(num_vertices):
        st.insert([i], filtration=-1e10)
    for x, y in zip(xs, ys):        
        st.insert([x, y], filtration=-1e10)
    for i in range(num_vertices):
        st.assign_filtration([i], values[i])
    st.make_filtration_non_decreasing()
    st.extend_filtration()
    LD = st.extended_persistence()
    dgmOrd0, dgmRel1, dgmExt0, dgmExt1 = LD[0], LD[1], LD[2], LD[3]
    dgmOrd0 = np.vstack([np.array([[ min(p[1][0],p[1][1]), max(p[1][0],p[1][1]) ]]) for p in dgmOrd0 if p[0] == 0]) if len(dgmOrd0) else np.empty([0,2])
    dgmRel1 = np.vstack([np.array([[ min(p[1][0],p[1][1]), max(p[1][0],p[1][1]) ]]) for p in dgmRel1 if p[0] == 1]) if len(dgmRel1) else np.empty([0,2])
    dgmExt0 = np.vstack([np.array([[ min(p[1][0],p[1][1]), max(p[1][0],p[1][1]) ]]) for p in dgmExt0 if p[0] == 0]) if len(dgmExt0) else np.empty([0,2])
    dgmExt1 = np.vstack([np.array([[ min(p[1][0],p[1][1]), max(p[1][0],p[1][1]) ]]) for p in dgmExt1 if p[0] == 1]) if len(dgmExt1) else np.empty([0,2])
    return dgmOrd0, dgmExt0, dgmRel1, dgmExt1



In [4]:
dir = 'data/MUTAG/mat'
files = os.listdir(dir)

n = len(files)
labels = np.zeros(n)
features = []

for i, file in enumerate(files):
    data = loadmat(dir + '/' +file)
    adj = np.array(data['A'], dtype = np.float32)
    L = csgraph.laplacian(adj, normed = True)
    egval, egvect = eigh(L)
    values = HKS(egvect,egval,1)
    dgmOrd0, dgmExt0, dgmRel1, dgmExt1 = extended_persistance(adj, values)
    features.append(dgmOrd0)
    features.append(dgmExt0)
    features.append(dgmRel1)
    features.append(dgmExt1)
    labels[i] = int(file[file.index('lb')+3])
    
features = tda.DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())]).fit_transform(features)
features = tda.Padding(use=True).fit_transform(features)

features = np.array(features)
features = np.reshape(features, (188,4,13,3))


labels = np.array(labels, dtype = np.int32)

        
# dataset = tf.data.Dataset.from_tensor_slices(features)

# rows, cols = np.where(adj == 10)
# edges = zip(rows.tolist(), cols.tolist())
# gr = nx.Graph()
# gr.add_edges_from(edges)
# nx.draw(gr, node_size=200)
# plt.show()
# plt.scatter(dgmOrd0[:,0], dgmOrd0[:,1])  
# plt.scatter(dgmExt0[:,0], dgmExt0[:,1])    
# plt.scatter(dgmRel1[:,0], dgmRel1[:,1])    
# plt.scatter(dgmExt1[:,0], dgmExt1[:,1])    

In [5]:
class PerslayModel(tf.keras.Model):
    def __init__(self):
        super(PerslayModel, self).__init__()

        self.weight =  [tf.Variable(name="weight", initial_value=rui(1.,1.)([10,10]), trainable = True) for _ in range(4)]
        self.layer_vars = tf.Variable(name ="vars", initial_value = rui(3.,3.)([4]), trainable = True)
        self.image_bnds = ((-0.001, 1.001), (-0.001, 1.001))
        self.image_size = (20, 20)
        self.fmodel = [tf.keras.Sequential([tf.keras.layers.Conv2D(10, 2, input_shape=(21,21,1)), tf.keras.layers.Flatten()]) for _ in range(4)]
        self.rho = tf.keras.Sequential([tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(16000,))])

        
    def call(self, inputs, training=False):
        list_v = []
        # 4 type of diagrams
        for nf in range(4):
            diag = inputs[:,nf,:,:]
            tensor_diag = diag[:,:,:-1]
            tensor_mask = diag[:,:,-1]
            
            # compute weights
            grid_shape = self.weight[nf].shape  
            indices = []
            for dim in range(2):
                [m, M] = (-0.001, 1.001)
                coords = tf.slice(tensor_diag, [0, 0, dim], [-1, -1, 1])
                ids = grid_shape[dim] * (coords - m)/(M - m)
                indices.append(tf.cast(ids, tf.int32))
            weight = tf.expand_dims(tf.gather_nd(params=self.weight[nf], indices=tf.concat(indices, axis=2)), -1)

            # compute layer
            num_pts = tensor_diag.shape[1]
            coords = [tf.range(start=self.image_bnds[i][0], limit=self.image_bnds[i][1], delta=(self.image_bnds[i][1] - self.image_bnds[i][0]) / self.image_size[i]) for i in range(2)]
            M = tf.meshgrid(*coords)
            t = tf.concat([tf.expand_dims(tens, 0) for tens in M], axis=0)
            bp_inp = tf.einsum("ijk,kl->ijl", tensor_diag, tf.constant(np.array([[1.,-1.],[0.,1.]], dtype=np.float32)))
            bc_inp = tf.reshape(bp_inp, [-1, num_pts, 2] + [1, 1])
            sg = self.layer_vars[nf]
            tensor_diag = tf.expand_dims(tf.math.exp(tf.math.reduce_sum(  -tf.math.square(bc_inp-t) / (2*tf.math.square(sg)),  axis=2)) / (2*np.pi*tf.math.square(sg)), -1)
            
            # Apply weight
            output_dim = len(tensor_diag.shape) - 2
            for _ in range(output_dim-1):
                weight = tf.expand_dims(weight, -1)
            tiled_weight = tf.tile(weight, [1, 1] + tensor_diag.shape[2:])
            tensor_diag = tf.math.multiply(tensor_diag, tiled_weight)

            # Apply mask
            for _ in range(output_dim):
                tensor_mask = tf.expand_dims(tensor_mask, -1)
            tiled_mask = tf.tile(tensor_mask, [1, 1] + tensor_diag.shape[2:])
            masked_layer = tf.math.multiply(tensor_diag, tiled_mask)

            # Permutation invariant operation sum
            vector = tf.math.reduce_sum(masked_layer, axis=1)
            
            # Convolution operation
            vector = self.fmodel[nf](vector)
            print('ok')
            list_v.append(vector)

        representations = tf.concat(values=list_v, axis=1)
        final_representations = self.rho(representations)

        return final_representations  

In [6]:
epochs    = 300
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, shuffle = True)


In [7]:
model = PerslayModel()
lr = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, decay_steps=20, decay_rate=0.5, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-4)
optimizer = tfa.optimizers.MovingAverage(optimizer, average_decay=0.9) 
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)


In [ ]:
history = model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=32, shuffle=True)
train_results = model.evaluate(x_train, y_train)
test_results = model.evaluate(x_test,  y_test)


Epoch 1/300
ok
ok
ok
ok
ok
ok
ok
ok


# TEST

In [ ]:
image_bnds = ((-0.001, 1.001), (-0.001, 1.001))
image_size     = (20, 20)

dimension_before = 2
coords = [tf.range(start=image_bnds[i][0], limit=image_bnds[i][1], delta=(image_bnds[i][1] - image_bnds[i][0]) / image_size[i]) for i in range(dimension_before)]
print(coords)
M = tf.meshgrid(*coords)
print('------------------')
print(len(M))
mu = tf.concat([tf.expand_dims(tens, 0) for tens in M], axis=0)
print('-------------------')
print(mu.shape)



In [ ]:
len(features[0])